In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_excel('/home/thomas/Nextcloud/CodeursEnLiberte/EMS/sports/extractions_denombrement.ods')

In [ ]:
df.shape

In [ ]:
entree_unitaire = df[~df['entrée unitaire'].isna()]

In [ ]:
entree_unitaire.shape

In [ ]:
sum(entree_unitaire.quantité * entree_unitaire['Pu entrée unitaire'])

In [ ]:
from openfisca_survey_manager.scenarios import AbstractSurveyScenario
from openfisca_france import CountryTaxBenefitSystem
from openfisca_france.model.base import Famille, FoyerFiscal, Menage
from openfisca_core import periods

In [ ]:
individu_df = pd.DataFrame({
    'famille_id': list(range(len(entree_unitaire))),
    'age': entree_unitaire.age,
    'qfrule': entree_unitaire.qfrule,
    'strasbourg_piscine_entree_unitaire': entree_unitaire.quantité
})
individu_df

In [ ]:
import sys
sys.path.append('../mobilite')

In [ ]:
from utils import *

In [ ]:
determine_qf(individu_df, "strasbourg_piscine_quotient_familial", qfrules_constant, lambda x: x)

In [ ]:
individu_df

In [ ]:
famille_df = pd.DataFrame({})
menage_df = pd.DataFrame({})
foyerfiscaux_df = pd.DataFrame({})

individu_df['famille_role_index'] = 0
individu_df['foyer_fiscal_id'] = individu_df.famille_id
individu_df['foyer_fiscal_role_index'] = 0
individu_df['menage_id'] = individu_df.famille_id
individu_df['menage_role_index'] = 0

In [ ]:
data = dict(input_data_frame_by_entity = dict(
individu=individu_df,
famille=famille_df,
menage=menage_df,
foyer_fiscal=foyerfiscaux_df))

In [ ]:
base_year = "2023"

base = CountryTaxBenefitSystem()
base.load_extension('openfisca_france_local')

In [ ]:
class StrasbourgSurveyScenario(AbstractSurveyScenario):
    def __init__(self, data = None,):
        super(StrasbourgSurveyScenario, self).__init__()

        self.year = base_year

        if 'input_data_frame_by_entity' in data:
            period = periods.period(self.year)
            dataframe_variables = set()
            for entity_dataframe in data['input_data_frame_by_entity'].values():
                if not isinstance(entity_dataframe, pd.DataFrame):
                    continue
                dataframe_variables = dataframe_variables.union(set(entity_dataframe.columns))
            self.used_as_input_variables = list(
                set(base.variables.keys()).intersection(dataframe_variables)
                )

        self.set_tax_benefit_systems(base)
        self.init_from_data(data = data)


In [ ]:
scenario = StrasbourgSurveyScenario(data = data)

In [ ]:
prix = scenario.simulation.calculate('strasbourg_piscine_prix_entree_unitaire', base_year)
res_prix = pd.DataFrame(data={
    "res":prix == entree_unitaire['Pu entrée unitaire'],
    "prix_input": entree_unitaire['Pu entrée unitaire'],
    "prix_output": prix})
res_prix[~res_prix.res]

In [ ]:
cout = scenario.simulation.calculate('strasbourg_piscine_cout_entree_unitaire', base_year)
res_cout = pd.DataFrame(data={
    "res":cout == entree_unitaire.euros,
    "prix_input": entree_unitaire.euros,
    "prix_output": cout}
                       )
res_cout[~res_cout.res]

In [ ]:
count = sum(entree_unitaire.quantité)

In [ ]:
sample_count = 1

In [ ]:
sample_ids = np.repeat(list(range(sample_count)), count)
sample_qfrule = np.tile(np.repeat(entree_unitaire.qfrule, entree_unitaire.quantité), sample_count)
sample_individu_df = pd.DataFrame({
    'sample_id': sample_ids,
    'famille_id': list(range(count * sample_count)),
    'strasbourg_piscine_quotient_familial': np.zeros(count * sample_count),
    'qfrule': sample_qfrule,
    'age': np.tile(np.repeat(entree_unitaire.age, entree_unitaire.quantité), sample_count),
    'strasbourg_piscine_entree_unitaire': np.ones(count * sample_count),
})
prix = np.tile(np.repeat(entree_unitaire['Pu entrée unitaire'], entree_unitaire.quantité), sample_count)

In [ ]:
determine_qf(sample_individu_df, 'strasbourg_piscine_quotient_familial', qfrules_constant, caf_to_fiscal= lambda x: x)

In [ ]:
sample_individu_df

In [ ]:
sample_famille_df = pd.DataFrame({})
sample_menage_df = pd.DataFrame({})
sample_foyerfiscaux_df = pd.DataFrame({})

sample_individu_df['famille_role_index'] = 0
sample_individu_df['foyer_fiscal_id'] = sample_individu_df.famille_id
sample_individu_df['foyer_fiscal_role_index'] = 0
sample_individu_df['menage_id'] = sample_individu_df.famille_id
sample_individu_df['menage_role_index'] = 0

sample_data = dict(input_data_frame_by_entity = dict(
individu=sample_individu_df,
famille=sample_famille_df,
menage=sample_menage_df,
foyer_fiscal=sample_foyerfiscaux_df))

sample_scenario = StrasbourgSurveyScenario(data = sample_data)

In [ ]:
sample_prix = sample_scenario.simulation.calculate('strasbourg_piscine_prix_entree_unitaire', base_year)
sum(sample_prix)

In [ ]:
sample_res_df = pd.DataFrame(data={
    "qfrule": sample_qfrule,
    "sample_id": sample_ids,
    "input_prix": prix,
    "output_prix": sample_prix
})
sample_res_df

In [ ]:
sample_sum = sample_res_df.groupby(["sample_id"]).sum()
sample_sum

In [ ]:
sample_sum_qfrule = sample_res_df.groupby(["qfrule"]).sum()
sample_sum_qfrule

In [ ]:
sample_sum.hist()

In [ ]:
463950/3

In [ ]:
463950/3*4/5*3 + 463950/3*1/5*5